In [1]:
!pip install torchsummary

In [2]:
# import the required packages
import torch
import argparse
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from torchvision.utils import save_image

from torchvision.utils import make_grid

from torch.utils.tensorboard import SummaryWriter
import datetime
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [3]:
import os
os.makedirs("SeaLake%d" % (1), exist_ok=True) 
cuda = True if torch.cuda.is_available() else False

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 16

In [5]:
image_shape = (3, 64, 64)
image_dim = int(np.prod(image_shape))
latent_dim = 100

In [6]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [7]:
train_transform = transforms.Compose([transforms.ToTensor()])#,transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
train_dataset = datasets.ImageFolder(root='../input/eurosattraindata/archive/RBG/train/SeaLake', transform=train_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [8]:
num_class = len(train_loader.dataset.classes)
print('Training samples: ', len(train_loader)*batch_size)
print('number of batches: ', len(train_loader))

In [9]:
def show_images(images):
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(images.detach(), nrow=22).permute(1, 2, 0))

def show_batch(dl):
    for images, _ in dl:
        show_images(images)
        break

In [10]:
show_batch(train_loader)

In [11]:
# Generator Model Class Definition     
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Block 1:input is Z, going into a convolution
            nn.ConvTranspose2d(latent_dim, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            # Block 2: input is (64 * 8) x 4 x 4
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            # Block 3: input is (64 * 4) x 8 x 8
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            # Block 4: input is (64 * 2) x 16 x 16
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # Block 5: input is (64) x 32 x 32
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: output is (3) x 64 x 64
            )
    def forward(self, input):
        output = self.main(input)
        return output

In [12]:
# Discriminator Model Class Definition
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Block 1: input is (3) x 64 x 64
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 2: input is (64) x 32 x 32
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 3: input is (64*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 4: input is (64*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 5: input is (64*8) x 4 x 4
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
            nn.Flatten()
            # Output: 1
            )
    def forward(self, input):
        output = self.main(input)
        return output

In [13]:
generator = Generator().to(device)
generator.apply(weights_init_normal)
print(generator)

In [14]:
discriminator = Discriminator().to(device)
discriminator.apply(weights_init_normal)
print(discriminator)

In [15]:
summary(generator, (100,1,1))

In [16]:
summary(discriminator, (3,64,64))

In [17]:
adversarial_loss = nn.BCELoss()

In [18]:
def generator_loss(fake_output, label):
    gen_loss = adversarial_loss(fake_output, label)
    #print(gen_loss)
    return gen_loss

In [ ]:
def discriminator_loss(output, label):
    disc_loss = adversarial_loss(output, label)
    return disc_loss

In [ ]:
fixed_noise = torch.randn(128, latent_dim, 1, 1, device=device)
real_label = 1
fake_label = 0

In [ ]:
learning_rate = 0.0002
G_optimizer = optim.Adam(generator.parameters(), lr = learning_rate, betas=(0.5, 0.999))
D_optimizer = optim.Adam(discriminator.parameters(), lr = learning_rate, betas=(0.5, 0.999))

In [ ]:

def train_and_generate():
    #right_path = 'SeaLake' + str(p)
    #left_path = './SeaLake'+ str(p)
    num_epochs = 300
    D_loss_plot, G_loss_plot = [], []
    for epoch in range(1, num_epochs+1): 

        D_loss_list, G_loss_list = [], []

        for index, (real_images, _) in enumerate(train_loader):
            D_optimizer.zero_grad()
            real_images = real_images.to(device)

            real_target = Variable(torch.ones(real_images.size(0)).to(device))
            fake_target = Variable(torch.zeros(real_images.size(0)).to(device))

            real_target = real_target.unsqueeze(1)
            fake_target = fake_target.unsqueeze(1)

            D_real_loss = discriminator_loss(discriminator(real_images), real_target)
            # print(discriminator(real_images))
            D_real_loss.backward()

            noise_vector = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)  
            noise_vector = noise_vector.to(device)

            generated_image = generator(noise_vector)
            output = discriminator(generated_image.detach())
            D_fake_loss = discriminator_loss(output,  fake_target)


            # train with fake
            D_fake_loss.backward()

            D_total_loss = D_real_loss + D_fake_loss
            D_loss_list.append(D_total_loss)

            #D_total_loss.backward()
            D_optimizer.step()

            # Train generator with real labels
            G_optimizer.zero_grad()
            G_loss = generator_loss(discriminator(generated_image), real_target)
            G_loss_list.append(G_loss)

            G_loss.backward()
            G_optimizer.step()


        print('Epoch: [%d/%d]: D_loss: %.3f, G_loss: %.3f' % (
                (epoch), num_epochs, torch.mean(torch.FloatTensor(D_loss_list)),\
                 torch.mean(torch.FloatTensor(G_loss_list))))

        D_loss_plot.append(torch.mean(torch.FloatTensor(D_loss_list)))
        G_loss_plot.append(torch.mean(torch.FloatTensor(G_loss_list)))

        if epoch == 300:
            for i in range(generated_image.shape[0]):
                save_image(generated_image[i], 'SeaLake1/%d.png' %i, normalize=True)
        else:
            pass

    #torch.save(generator.state_dict(), left_path + '/generator_epoch_%d.pth' % (epoch))
    #torch.save(discriminator.state_dict(), left_path + '/discriminator_epoch_%d.pth' % (epoch))

In [ ]:
train_and_generate()